In [2]:
%pwd

'/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/src/analysis/round3'

In [3]:
%cd ../../../data/round3/days/

/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/data/round3/days


In [4]:
import pandas as pd
import numpy as np

In [5]:
df_mid_0 = pd.read_csv('mid_price_day0.csv')
df_mid_1 = pd.read_csv('mid_price_day1.csv')
df_mid_2 = pd.read_csv('mid_price_day2.csv')


In [7]:
#补充时间序列
df_mid_0['T'] = (7 - df_mid_0['timestamp'] * 0.0001 / 100  ) / 365
df_mid_1['T'] = (6 - df_mid_1['timestamp'] * 0.0001 / 100  ) / 365
df_mid_2['T'] = (5 - df_mid_2['timestamp'] * 0.0001 / 100  ) / 365

In [20]:
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import numpy as np

def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def implied_volatility(S, K, T, r, market_price):
    loss_fn = lambda sigma: (bs_call_price(S, K, T, r, sigma) - market_price)**2
    result = minimize_scalar(loss_fn, bounds=(1e-4, 3.0), method='bounded')
    return result.x

def calculate_iv(df):
    df = df.copy()
    strick_prices = [9500, 9750, 10000, 10250, 10500]
    for strick_price in strick_prices:
        df['iv_' + str(strick_price)] = df.apply(lambda row: implied_volatility(
            row['VOLCANIC_ROCK'], strick_price, row['T'], 0, row['VOLCANIC_ROCK_VOUCHER_' + str(strick_price)]), axis=1)
    return df

from sklearn.linear_model import LinearRegression

def weighted_linear_predict(y, window=10):
    if len(y) < window:
        return np.nan
    y = np.array(y[-window:])
    X = np.arange(window).reshape(-1, 1)
    weights = np.exp(-0.1 * (window - np.arange(window)))  # 越近权重越大
    model = LinearRegression()
    model.fit(X, y, sample_weight=weights)
    return model.predict([[window]])[0] 

def predict_fair_price(df):
    df = df.copy()
    strick_prices = [9500, 9750, 10000, 10250, 10500]
    for strick_price in strick_prices:
        df['predicted_iv_' + str(strick_price)] = df['iv_' + str(strick_price)].rolling(window=10).apply(weighted_linear_predict, raw=False)
        df['fair_price_' + str(strick_price)] = df.apply(lambda row: bs_call_price(
            row['VOLCANIC_ROCK'], strick_price, row['T'], 0, row['predicted_iv_' + str(strick_price)]), axis=1)
        df['delta_' + str(strick_price)] = df['fair_price_' + str(strick_price)] - df['VOLCANIC_ROCK_VOUCHER_' + str(strick_price)]
    df.dropna(inplace=True)
    return df

In [21]:
def preprocess(df):
    df = df.copy()
    df = calculate_iv(df)
    df = predict_fair_price(df)
    return df

In [ ]:
df_mid_0 = preprocess(df_mid_0)
df_mid_1 = preprocess(df_mid_1)
df_mid_2 = preprocess(df_mid_2)
df_mid_0.to_csv('day0.csv')
df_mid_1.to_csv('day1.csv')
df_mid_2.to_csv('day2.csv')